In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('sun_and_cloud.csv',encoding='cp949')
# data.drop(columns=['지면온도(°C)','시정(10m)'],inplace=True)

print(data.info())
print('----------------------')
print(data.max())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26304 entries, 0 to 26303
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   시간           26304 non-null  int64  
 1   일조(hr)       26304 non-null  float64
 2   일사(MJ/m2)    26304 non-null  float64
 3   전운량(10분위)    26304 non-null  float64
 4   중하층운량(10분위)  26304 non-null  float64
 5   지면온도(°C)     26304 non-null  float64
 6   시정(10m)      26304 non-null  int64  
 7   전력거래량        26304 non-null  float64
dtypes: float64(6), int64(2)
memory usage: 1.6 MB
None
----------------------
시간               23.000000
일조(hr)            1.000000
일사(MJ/m2)         3.780000
전운량(10분위)        10.000000
중하층운량(10분위)      10.000000
지면온도(°C)         62.400000
시정(10m)        5000.000000
전력거래량            10.807822
dtype: float64


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
result = scaler.fit_transform(data)
df_scld= pd.DataFrame(data=result,columns=data.columns)

In [ ]:
x_train = df_scld.iloc[:25560,:-1] # 18년 ~ 20년 11월 : 25560 --> Train & Validation (24096 ==> 20년 10월 1일 00시)
y_train = df_scld.iloc[:25560,-1] # 18년 ~ 20년 11월 --> Train & Validation

x_pred = df_scld.iloc[25560:,:-1] # 20년 12월- 예측용 변수 --> Preidction
y_pred = df_scld.iloc[25560:,-1] # 20년 12월 - 예측용 정답 --> Preidction


# --- Ensemble 을 위해서 np.array 형태로 data type 전환해보기
x_train = np.array(x_train)
y_train = np.array(y_train)

x_pred = np.array(x_pred)
y_pred = np.array(y_pred)

print(x_train,'\n',x_pred,'\n',y_train,'\n',y_pred)
print('------------------------------------------------------------------------------------------------------------------------')
print(x_train.shape,'\n',x_pred.shape,'\n',y_train.shape,'\n',y_pred.shape)

[[0.         0.         0.         ... 0.         0.15       0.26823388]
 [0.04347826 0.         0.         ... 0.         0.14342105 0.39722725]
 [0.08695652 0.         0.         ... 0.         0.13947368 0.39722725]
 ...
 [0.91304348 0.         0.         ... 0.         0.16973684 0.39722725]
 [0.95652174 0.         0.         ... 0.         0.16052632 0.39722725]
 [1.         0.         0.         ... 0.         0.15657895 0.39722725]] 
 [[0.         0.         0.         ... 0.         0.15526316 0.39722725]
 [0.04347826 0.         0.         ... 0.         0.15394737 0.39722725]
 [0.08695652 0.         0.         ... 0.         0.15131579 0.39722725]
 ...
 [0.91304348 0.         0.         ... 0.         0.10394737 0.39722725]
 [0.95652174 0.         0.         ... 0.         0.09868421 0.39722725]
 [1.         0.         0.         ... 0.         0.09342105 0.39722725]] 
 [0.         0.         0.         ... 0.00438201 0.         0.        ] 
 [0.00000000e+00 0.00000000e+00 0.0

In [ ]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],1)
#x_val = x_val.reshape(x_val.shape[0],x_val.shape[1],1)
x_pred = x_pred.reshape(x_pred.shape[0],x_pred.shape[1],1)

#print(x_train.shape,'\n',x_val.shape,'\n',x_pred.shape)
print(x_train.shape,'\n',x_pred.shape)

feature_num = len(x_train[0])
feature_num

(25560, 7, 1) 
 (744, 7, 1)


7

In [ ]:
from tensorflow.keras import models,layers
gru= models.Sequential() # 모델
gru.add(layers.GRU(units=32,return_sequences=True,  activation='tanh', input_shape=(feature_num,1)))
gru.add(layers.Dropout(rate=0.2))
gru.add(layers.GRU(units=64,return_sequences=True, activation='tanh'))
gru.add(layers.Dropout(rate=0.2))
gru.add(layers.GRU(64, activation='tanh')) 
gru.add(layers.Dropout(rate=0.2))
gru.add(layers.Dense(1)) 
gru.summary()
gru.compile(loss='mse', optimizer='adam', metrics=['mae']) 

from keras.models import Sequential 
from keras.layers import Dense, SimpleRNN , Dropout
rnn = Sequential()
rnn.add(layers.SimpleRNN(units=32,return_sequences=True,  activation='tanh', input_shape=(feature_num,1)))
rnn.add(layers.Dropout(rate=0.2))
rnn.add(layers.SimpleRNN(units=64,return_sequences=True, activation='tanh'))
rnn.add(layers.Dropout(rate=0.2))
rnn.add(layers.SimpleRNN(64, activation='tanh')) 
rnn.add(layers.Dropout(rate=0.2))
rnn.add(layers.Dense(1)) 
rnn.summary()
rnn.compile(optimizer = 'adam', loss = 'mse' , metrics=['mae'])

from tensorflow.keras import models, layers
lstm = models.Sequential() # 모델
lstm.add(layers.LSTM(units=32,return_sequences=True,  activation='tanh', input_shape=(feature_num,1)))
lstm.add(layers.Dropout(rate=0.2))
lstm.add(layers.LSTM(units=64,return_sequences=True, activation='tanh'))
lstm.add(layers.Dropout(rate=0.2))
lstm.add(layers.LSTM(64, activation='tanh')) 
lstm.add(layers.Dropout(rate=0.2))
lstm.add(layers.Dense(1)) 
lstm.summary()
lstm.compile(loss='mse', optimizer='adam', metrics=['mae']) 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 7, 32)             3360      
                                                                 
 dropout (Dropout)           (None, 7, 32)             0         
                                                                 
 gru_1 (GRU)                 (None, 7, 64)             18816     
                                                                 
 dropout_1 (Dropout)         (None, 7, 64)             0         
                                                                 
 gru_2 (GRU)                 (None, 64)                24960     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 6

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.ensemble import VotingRegressor

Epoch = 200 #------------------------------------------ 원하는대로 바꿔도 됨.

est_gru = KerasRegressor(build_fn=lambda:gru, epochs=Epoch-50, verbose=1, validation_split=0.2)
est_gru._estimator_type="regressor"
est_rnn = KerasRegressor(build_fn=lambda:rnn, epochs=Epoch, verbose=1, validation_split=0.2)
est_rnn._estimator_type="regressor"
est_lstm = KerasRegressor(build_fn=lambda:lstm, epochs=Epoch+50, verbose=1, validation_split=0.2)
est_lstm._estimator_type="regressor"

vote = VotingRegressor(estimators=[('gru', est_gru), ('rnn', est_rnn), ('lstm', est_lstm)])
#vote = VotingRegressor(estimators=[('gru', gru), ('rnn', rnn), ('lstm', lstm)])
history = vote.fit(x_train,y_train)

In [ ]:
Vote_Pred = vote.predict(x_pred)

In [ ]:
max_val = data['전력거래량'].max()
Vote_Pred_Rescale = max_val*Vote_Pred
print(Vote_Pred_Rescale)

In [ ]:
for i in range (0,len(Vote_Pred_Rescale)):
  if Vote_Pred_Rescale[i] < 0 :
    Vote_Pred_Rescale[i] = 0

print(Vote_Pred_Rescale)

In [ ]:
import matplotlib.pyplot as plt

# 17520 부터 2020년 data

for t in range (0,10): # 10개씩 plot figure 보고싶음
  date= t + 20  #-------- 보고싶은 구간 반영하여 0~21 까지 수동으로 더해주기
  Ans_date = 25584+date*24 # --- Answer 범위가 range-24부터 시작하니까 (21888부터 20년 7월) (25560부터 20년 12월)
  
  x=np.array(range(24))
  Y = Vote_Pred_Rescale[24*date:24*date+24]
  Answer = pd.Series(data[Ans_date-24:Ans_date]['전력거래량']).array
  #Answer = pd.Series(data[Ans_date-24:Ans_date]['전력거래량']).array
  
  plt.figure()
  plt.plot(x,Y,'bo:')
  plt.plot(x,Answer,'r*-')
  plt.legend(['Vote','Real'])

In [ ]:
from sklearn.metrics import mean_absolute_error

voting_mae = mean_absolute_error(vote.predict(x_pred), y_pred)
print('Voting Regressor mae :', max_val*voting_mae)